# Bài tập: Trích xuất và phân tích tên riêng từ bài báo tiếng Việt

## Yêu cầu:
1. Tách từ và lưu lại các tên riêng trong mỗi bài báo
2. Nhập vào 1 từ khóa, tìm xem tên riêng đó xuất hiện trong các bài báo nào
3. In ra top 10-30 các tên riêng có tần số xuất hiện nhiều nhất
4. Tìm và in ra các bộ tên riêng có liên quan đến nhau (xuất hiện đồng thời trong nhiều bài báo nhất)

## 1. Đọc dữ liệu và trích xuất tên riêng

In [2]:
import os
import json
import re
from collections import defaultdict
from itertools import combinations

# Đường dẫn đến thư mục chứa bài báo
ARTICLE_PATH = "Article_Crawl"

# Danh sách các từ phổ biến KHÔNG phải tên riêng (stopwords)
# Các từ viết hoa đầu câu nhưng không phải tên riêng
STOPWORDS = {
    "Theo", "Trong", "Với", "Từ", "Đến", "Khi", "Nếu", "Như", "Sau", "Trước",
    "Tại", "Về", "Và", "Hoặc", "Nhưng", "Tuy", "Vì", "Do", "Bởi", "Để",
    "Còn", "Cũng", "Đã", "Đang", "Sẽ", "Được", "Bị", "Có", "Là", "Này",
    "Đó", "Những", "Các", "Một", "Nhiều", "Ít", "Rất", "Quá", "Hơn", "Nhất",
    "Thì", "Mà", "Cho", "Của", "Ở", "Ra", "Vào", "Lên", "Xuống", "Nên",
    "Hay", "Thế", "Sao", "Vậy", "Đây", "Kia", "Nào", "Gì", "Ai", "Đâu",
    "Bao", "Mấy", "Sự", "Việc", "Điều", "Cách", "Lúc", "Khi", "Nơi", "Chỗ",
    "Tôi", "Bạn", "Anh", "Chị", "Em", "Ông", "Bà", "Cô", "Chú", "Bác",
    "Họ", "Chúng", "Mình", "Ta", "Người", "Ai", "Đều", "Rồi", "Lại", "Vẫn",
    "Thêm", "Bớt", "Hết", "Xong", "Mới", "Cũ", "Tốt", "Xấu", "Lớn", "Nhỏ",
    "Cao", "Thấp", "Dài", "Ngắn", "Rộng", "Hẹp", "Sâu", "Nông", "Nhanh", "Chậm",
    "Tuy", "Nhiên", "Song", "Thực", "Chất", "Đặc", "Biệt", "Riêng", "Chung",
    "Đồng", "Thời", "Hiện", "Tại", "Quá", "Khứ", "Tương", "Lai", "Nay", "Xưa",
    "Liên", "Quan", "Ngay", "Đúng", "Sai", "Phải", "Trái", "Đông", "Tây", "Nam", "Bắc",
    "Trên", "Dưới", "Ngoài", "Cùng", "Khác", "Giống", "Đơn", "Đôi", "Ba", "Bốn",
    "Năm", "Sáu", "Bảy", "Tám", "Chín", "Mười", "Trăm", "Nghìn", "Triệu", "Tỷ",
    "Phần", "Hàng", "Loại", "Kiểu", "Dạng", "Thể", "Diện", "Tích", "Số", "Lượng",
    "Chính", "Phó", "Thứ", "Đại", "Tiểu", "Siêu", "Cực", "Toàn", "Bán", "Nửa",
    "Ảnh", "Video", "Hình", "Clip", "Nguồn", "Theo", "Sáng", "Chiều", "Tối", "Đêm",
    "Thứ", "Ngày", "Tháng", "Quý", "Tuần", "Giờ", "Phút", "Giây", "Hôm", "Mai",
    "Bên", "Phía", "Hướng", "Góc", "Cạnh", "Giữa", "Cuối", "Đầu", "Giữa",
    "The", "Of", "And", "In", "To", "For", "Is", "On", "At", "By",
    "Đảng", "Nhà", "Nước", "Quốc", "Gia", "Dân", "Tộc", "Hội", "Đoàn", "Tổ",
    "Sở", "Cục", "Vụ", "Ban", "Ngành", "Bộ", "Phòng", "Khoa", "Viện", "Trường"
}

print("Danh sách stopwords gồm", len(STOPWORDS), "từ")

Danh sách stopwords gồm 231 từ


In [3]:
def extract_proper_names(text):
    """
    Trích xuất tên riêng từ văn bản tiếng Việt.
    
    Thuật toán đơn giản:
    - Tìm các cụm từ viết hoa liên tiếp (2-5 từ)
    - Loại bỏ các từ trong stopwords
    - Tên riêng tiếng Việt thường có 2-4 từ viết hoa
    
    Returns:
        set: Tập hợp các tên riêng tìm được
    """
    if not text:
        return set()
    
    # Pattern để tìm cụm từ viết hoa (tên riêng tiếng Việt)
    # Mỗi từ viết hoa bắt đầu bằng chữ cái Unicode in hoa
    # Ví dụ: "Nguyễn Văn A", "Hồ Chí Minh", "Tô Lâm"
    pattern = r'\b([A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ][a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]*(?:\s+[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ][a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]*){1,4})\b'
    
    matches = re.findall(pattern, text)
    
    proper_names = set()
    for match in matches:
        name = match.strip()
        words = name.split()
        
        # Loại bỏ nếu tất cả các từ đều là stopwords
        non_stop_words = [w for w in words if w not in STOPWORDS]
        
        # Chỉ giữ lại tên có ít nhất 2 từ và có từ không phải stopword
        if len(words) >= 2 and len(non_stop_words) >= 1:
            # Loại bỏ stopwords ở đầu và cuối
            while words and words[0] in STOPWORDS:
                words = words[1:]
            while words and words[-1] in STOPWORDS:
                words = words[:-1]
            
            if len(words) >= 2:
                clean_name = ' '.join(words)
                # Loại bỏ các tên quá dài (có thể là câu viết hoa đầu)
                if len(clean_name) <= 50:
                    proper_names.add(clean_name)
    
    return proper_names

# Test
test_text = "Tổng Bí thư Tô Lâm và Chủ tịch nước Lương Cường đã gặp Thủ tướng Phạm Minh Chính."
print("Test:", extract_proper_names(test_text))

Test: {'Tô Lâm', 'Lương Cường', 'Tổng Bí', 'Phạm Minh'}


In [4]:
def load_article(filepath):
    """
    Đọc 1 bài báo từ file JSON.
    
    Returns:
        tuple: (text, filename) hoặc (None, filename) nếu lỗi
    """
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)
            # Kết hợp tiêu đề, tóm tắt và nội dung
            text = ' '.join(filter(None, [
                data.get('Subject', ''),
                data.get('Summary', ''),
                data.get('Content', '')
            ]))
            return text, os.path.basename(filepath)
    except:
        return None, os.path.basename(filepath)

def get_all_article_files():
    """
    Lấy danh sách tất cả các file bài báo.
    
    Returns:
        list: Danh sách đường dẫn đầy đủ đến các file
    """
    files = []
    for source_dir in os.listdir(ARTICLE_PATH):
        source_path = os.path.join(ARTICLE_PATH, source_dir)
        if os.path.isdir(source_path):
            for filename in os.listdir(source_path):
                if filename.endswith('.txt'):
                    files.append(os.path.join(source_path, filename))
    return files

# Đếm số file
all_files = get_all_article_files()
print(f"Tổng số bài báo: {len(all_files)}")

Tổng số bài báo: 34168


In [5]:
# CẤU TRÚC DỮ LIỆU CHÍNH
# 1. name_to_articles: Dict[tên_riêng] -> Set[tên_file]
#    Dùng để tìm kiếm bài báo chứa 1 tên riêng
#
# 2. article_to_names: Dict[tên_file] -> Set[tên_riêng]
#    Dùng để tìm các tên riêng đồng xuất hiện

name_to_articles = defaultdict(set)  # tên_riêng -> {các file chứa tên này}
article_to_names = {}                 # tên_file -> {các tên riêng trong bài}

print("Bắt đầu xử lý các bài báo...")
print("(Có thể mất vài phút)")

# Xử lý từng bài báo
processed = 0
for filepath in all_files:
    text, filename = load_article(filepath)
    
    if text:
        # Trích xuất tên riêng
        names = extract_proper_names(text)
        
        # Lưu vào cấu trúc dữ liệu
        article_to_names[filename] = names
        
        for name in names:
            name_to_articles[name].add(filename)
    
    processed += 1
    if processed % 5000 == 0:
        print(f"Đã xử lý {processed}/{len(all_files)} bài...")

print(f"\nHoàn thành! Đã xử lý {processed} bài báo")
print(f"Tổng số tên riêng tìm được: {len(name_to_articles)}")

Bắt đầu xử lý các bài báo...
(Có thể mất vài phút)
Đã xử lý 5000/34168 bài...
Đã xử lý 10000/34168 bài...
Đã xử lý 15000/34168 bài...
Đã xử lý 20000/34168 bài...
Đã xử lý 25000/34168 bài...
Đã xử lý 30000/34168 bài...

Hoàn thành! Đã xử lý 34168 bài báo
Tổng số tên riêng tìm được: 82742


## 2. Tìm kiếm bài báo chứa tên riêng

In [6]:
def search_name(keyword):
    """
    Tìm các bài báo chứa tên riêng khớp với từ khóa.
    
    Thuật toán: Tìm kiếm tuyến tính O(n) với n là số tên riêng
    - Duyệt qua tất cả tên riêng
    - Kiểm tra nếu keyword là substring (không phân biệt hoa/thường)
    
    Args:
        keyword: Từ khóa tìm kiếm
        
    Returns:
        dict: {tên_riêng: [danh sách file]}
    """
    keyword_lower = keyword.lower()
    results = {}
    
    for name, articles in name_to_articles.items():
        if keyword_lower in name.lower():
            results[name] = list(articles)
    
    return results

# Demo tìm kiếm
def demo_search(keyword):
    print(f"\n🔍 Tìm kiếm: '{keyword}'")
    print("=" * 50)
    
    results = search_name(keyword)
    
    if not results:
        print("Không tìm thấy kết quả nào.")
        return
    
    total_articles = 0
    for name, articles in sorted(results.items(), key=lambda x: -len(x[1])):
        print(f"\n📌 {name} ({len(articles)} bài báo):")
        for article in articles[:5]:  # Hiển thị tối đa 5 file
            print(f"   - {article}")
        if len(articles) > 5:
            print(f"   ... và {len(articles) - 5} bài khác")
        total_articles += len(articles)
    
    print(f"\n📊 Tổng: {len(results)} tên riêng khớp, {total_articles} lượt xuất hiện")

In [7]:
# Thử tìm kiếm một số tên
demo_search("Hồ Chí Minh")


🔍 Tìm kiếm: 'Hồ Chí Minh'

📌 Hồ Chí Minh (2108 bài báo):
   - 2025-05-19_soha.vn_11-58-30.txt
   - 2025-05-19_thanhnien.vn_14-01-42.txt
   - 2025-05-19_nhandan.vn_12-19-43.txt
   - 2025-05-19_phapluatxahoi.kinhtedothi.vn_12-22-47.txt
   - 2025-05-19_nhandan.vn_12-44-37.txt
   ... và 2103 bài khác

📌 Hồ Chí Minh Viếng (16 bài báo):
   - 2025-05-19_nhandan.vn_12-01-32.txt
   - 2025-05-19_nhandan.vn_12-01-22.txt
   - 2025-05-19_nhandan.vn_12-27-26.txt
   - 2025-05-19_nhandan.vn_12-14-48.txt
   - 2025-05-19_nhandan.vn_12-01-35.txt
   ... và 11 bài khác

📌 Hồ Chí Minh Xúc (15 bài báo):
   - 2025-05-19_nhandan.vn_12-01-32.txt
   - 2025-05-19_nhandan.vn_12-01-22.txt
   - 2025-05-19_nhandan.vn_12-14-48.txt
   - 2025-05-19_nhandan.vn_12-01-35.txt
   - 2025-05-19_nhandan.vn_12-00-33.txt
   ... và 10 bài khác

📌 Đường Hồ Chí Minh (14 bài báo):
   - 2025-05-19_nhandan.vn_13-08-46.txt
   - 2025-05-19_vovgiaothong.vn_12-14-41.txt
   - 2025-05-19_cadn.com.vn_11-57-56.txt
   - 2025-05-19_vnexpress.ne

In [ ]:
# Tìm kiếm tương tác - người dùng nhập từ khóa
keyword = input("Nhập tên riêng cần tìm: ")
demo_search(keyword)

## 3. Top tên riêng xuất hiện nhiều nhất

In [8]:
def get_top_names(n=30):
    """
    Lấy top n tên riêng xuất hiện trong nhiều bài báo nhất.
    
    Thuật toán: Sắp xếp O(m log m) với m là số tên riêng
    
    Args:
        n: Số lượng tên riêng cần lấy
        
    Returns:
        list: [(tên_riêng, số_bài_báo), ...]
    """
    # Tạo danh sách (tên, số bài) và sắp xếp giảm dần
    name_counts = [(name, len(articles)) for name, articles in name_to_articles.items()]
    name_counts.sort(key=lambda x: -x[1])
    
    return name_counts[:n]

# In top 30 tên riêng
print("\n🏆 TOP 30 TÊN RIÊNG XUẤT HIỆN NHIỀU NHẤT")
print("=" * 60)
print(f"{'Hạng':<6}{'Tên riêng':<35}{'Số bài báo':>15}")
print("-" * 60)

top_names = get_top_names(30)
for i, (name, count) in enumerate(top_names, 1):
    print(f"{i:<6}{name:<35}{count:>15}")


🏆 TOP 30 TÊN RIÊNG XUẤT HIỆN NHIỀU NHẤT
Hạng  Tên riêng                               Số bài báo
------------------------------------------------------------
1     Hà Nội                                        6130
2     Hồ Chí Minh                                   2108
3     Đà Nẵng                                       1691
4     Nhật Bản                                      1547
5     Thái Lan                                      1129
6     Sài Gòn                                        879
7     Ấn Độ                                          840
8     Thanh Hóa                                      757
9     Bình Dương                                     746
10    Nghệ An                                        734
11    Tổng Bí                                        662
12    Thủ Đức                                        629
13    Mỹ Donald Trump                                612
14    Tô Lâm                                         580
15    Bình Định                            

## 4. Tìm các cặp tên riêng liên quan (đồng xuất hiện)

In [10]:
def find_cooccurring_names(min_cooccurrence=10, top_n=30):
    """
    Tìm các cặp tên riêng xuất hiện đồng thời trong nhiều bài báo.
    
    Thuật toán:
    - Duyệt qua từng bài báo
    - Với mỗi bài, tạo tất cả các cặp tên riêng (combinations)
    - Đếm số lần xuất hiện của mỗi cặp
    
    Độ phức tạp: O(A * N^2) với A là số bài, N là số tên trung bình mỗi bài
    
    Args:
        min_cooccurrence: Số lần đồng xuất hiện tối thiểu
        top_n: Số cặp cần trả về
        
    Returns:
        list: [((tên1, tên2), số_lần_đồng_xuất_hiện), ...]
    """
    # Đếm số lần đồng xuất hiện của mỗi cặp
    pair_counts = defaultdict(int)
    
    # Chỉ xét các tên xuất hiện ít nhất min_cooccurrence lần
    # để giảm số cặp phải xét
    frequent_names = {name for name, articles in name_to_articles.items() 
                      if len(articles) >= min_cooccurrence}
    
    print(f"Đang phân tích {len(frequent_names)} tên riêng phổ biến...")
    
    processed = 0
    for filename, names in article_to_names.items():
        # Lọc chỉ giữ các tên phổ biến
        filtered_names = names & frequent_names
        
        # Tạo tất cả các cặp và đếm
        for name1, name2 in combinations(sorted(filtered_names), 2):
            pair_counts[(name1, name2)] += 1
        
        processed += 1
        if processed % 10000 == 0:
            print(f"  Đã xử lý {processed} bài...")
    
    # Sắp xếp theo số lần đồng xuất hiện giảm dần
    sorted_pairs = sorted(pair_counts.items(), key=lambda x: -x[1])
    
    # Lọc và trả về top n
    result = [(pair, count) for pair, count in sorted_pairs 
              if count >= min_cooccurrence][:top_n]
    
    return result

In [11]:
# Tìm và hiển thị các cặp tên liên quan
cooccurring_pairs = find_cooccurring_names(min_cooccurrence=5, top_n=30)

print("\n🔗 TOP 30 CẶP TÊN RIÊNG ĐỒNG XUẤT HIỆN NHIỀU NHẤT")
print("=" * 80)
print(f"{'Hạng':<6}{'Tên 1':<25}{'Tên 2':<25}{'Số bài chung':>15}")
print("-" * 80)

for i, ((name1, name2), count) in enumerate(cooccurring_pairs, 1):
    print(f"{i:<6}{name1:<25}{name2:<25}{count:>15}")

Đang phân tích 6927 tên riêng phổ biến...
  Đã xử lý 10000 bài...
  Đã xử lý 20000 bài...
  Đã xử lý 30000 bài...

🔗 TOP 30 CẶP TÊN RIÊNG ĐỒNG XUẤT HIỆN NHIỀU NHẤT
Hạng  Tên 1                    Tên 2                       Số bài chung
--------------------------------------------------------------------------------
1     Hà Nội                   Hồ Chí Minh                          806
2     Hà Nội                   Đà Nẵng                              624
3     Tô Lâm                   Tổng Bí                              560
4     Hà Nội                   Thanh Hóa                            393
5     Hà Nội                   Nhật Bản                             310
6     Hà Nội                   Nghệ An                              307
7     Hồ Chí Minh              Đà Nẵng                              274
8     Hồ Chí Minh              Tổng Bí                              263
9     Hà Nội                   Thái Lan                             253
10    Bình Dương               Hà N

## 5. Tổng kết

### Các cấu trúc dữ liệu đã sử dụng:
1. **Dictionary (defaultdict)**: Lưu trữ mapping tên riêng -> bài báo với O(1) lookup
2. **Set**: Lưu trữ tập hợp tên riêng trong mỗi bài, hỗ trợ phép giao nhanh
3. **List + Sort**: Sắp xếp để tìm top N

### Các thuật toán:
1. **Trích xuất tên riêng**: Sử dụng regex pattern matching
2. **Tìm kiếm**: Tìm kiếm tuyến tính với substring matching O(n*m)
3. **Top N**: Sắp xếp O(n log n) 
4. **Đồng xuất hiện**: Duyệt + combinations O(A * N^2)

In [12]:
# Thống kê tổng quan
print("\n📊 THỐNG KÊ TỔNG QUAN")
print("=" * 40)
print(f"Tổng số bài báo đã xử lý: {len(article_to_names)}")
print(f"Tổng số tên riêng unique: {len(name_to_articles)}")

# Số tên trung bình mỗi bài
avg_names = sum(len(names) for names in article_to_names.values()) / len(article_to_names)
print(f"Số tên riêng trung bình/bài: {avg_names:.1f}")

# Tên xuất hiện nhiều nhất
if top_names:
    print(f"Tên xuất hiện nhiều nhất: {top_names[0][0]} ({top_names[0][1]} bài)")


📊 THỐNG KÊ TỔNG QUAN
Tổng số bài báo đã xử lý: 31511
Tổng số tên riêng unique: 82742
Số tên riêng trung bình/bài: 7.6
Tên xuất hiện nhiều nhất: Hà Nội (6130 bài)
